In [1]:
"""Example app pour chercher des adjectifs sur Wiktionary"""
import mechanicalsoup
import re
baseURL="https://fr.wiktionary.org"

In [2]:
browser = mechanicalsoup.Browser()

### Traitement d'une page Adjectif
Sur la page d'un mot venant d'une des pages de la liste des adjectifs, il y a une partie qui définit l'adjectif. Cette partie commence avec le mot Adjectif. On cherche la structure qui contient 

In [3]:
def getAdj(url):
    # print url
    adjPage=browser.get(baseURL+url)
    try:
        divAdj=adjPage.soup.find(string="Adjectif")
        pAdj=divAdj.parent.parent.find_next("p")
        api=pAdj.find(title="Prononciation API").text
        try:
            translit=pAdj.find(title="Wiktionnaire:Conventions/arabe/Translittération").text
        except:
            translit=""
        def is_abreg(chaine):
            return u"écriture abrégée" in chaine
        try:
            simple=pAdj.find(string=is_abreg).find_next("a").text
        except:
            simple=""
        # print api, translit, simple
        try:
            trad=[]
            for li in pAdj.find_next("ol").findAll("li"):
                trad.append(li.text)
        except:
            trad=[""]
    except:
        api=""
        translit=""
        simple=""
        trad=[""]
    return (api,translit,simple," --- ".join(trad))

In [4]:
def getAdjList(url):
    fullPage=browser.get(baseURL+url)
    divList=fullPage.soup.find(id="mw-pages")
    # l'url relative de la page suivante est dans la balise <a> dont le texte est "page suivante"
    # si il n'y a pas d'url, la collecte est terminée => nextURL=False
    try:
        nextURL=divList.find("a",string="page suivante").get("href")
    except:
        nextURL=False
    # la structure qui contient la liste des adjectifs a l'id "mw-content-ltr"
    listeAdj=divList.find(class_="mw-content-ltr")
    # les adjectifs sont dans les balises <li> à l'intérieur du <a>
    adjectifsLoc={}
    for a in listeAdj.findAll("a"):
        adjectifsLoc[a.text]=getAdj(a.get("href"))
    return adjectifsLoc,nextURL

In [5]:
adjectifsWiki,nextPage=getAdjList("/wiki/Catégorie:Adjectifs_en_arabe")
for i in range(10):
    print "page",i
    if nextPage:
        adjectifsPage,nextPage=getAdjList(nextPage)
        adjectifsWiki.update(adjectifsPage)
    else:
        break
len(adjectifsWiki)

page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7


1471

### Ecrire le TSV

In [7]:
import codecs
with codecs.open("ADJ-Wiktionary.tsv","w",encoding="utf8") as outFile:
    for k,v in adjectifsWiki.iteritems():
        outFile.write(k)
        outFile.write("\t")
        outFile.write(v[0])        
        outFile.write("\t")
        outFile.write(v[1])        
        outFile.write("\t")
        outFile.write(v[2])
        outFile.write("\t")
        outFile.write(v[3].replace("\n"," "))
        outFile.write("\n")        

# FIN